#### <h1> <center> ENSF 519.01 Programming Fundamentals for Data Engineers </center></h1>
<h2> <center> Assignment 5: Numpy and Pandas (100 marks)</center></h2>

<center>
<div class="alert alert-block alert-info">
Updated Due: Sunday 28 Oct Midnight. To be submitted on D2L.
</div></center>


Edit this file and write your solutions to the problems in sections specified with `# Your solution goes here`. Test your code and when you were done, download this notebook as an `.ipynb` file and submit it to D2L. To get this file, in Jupyter notebook you can go to File -> Download as -> Notebook(.ipynb)

# Working with Numbers

In this assignment, as a data scientist you are going to learn how to efficiently work with numbers and perform the operations faster using `numpy` and `pandas` libraries. 

### Before You start: 
##### Make Sure :
* Numpy and pandas are already installed with the Anaconda stack. If you installed jupyter directly, make sure that you have them installed:<br> 
    `import numpy
    numpy.__version__`

* Download data.csv and put it somewhere in your repository to be accessible in you code (for ease of access put it in the same directory of you notebook file)

<div class="alert alert-block alert-info">
<b>Tip:</b> Use `numpy ?` to read numpy documentation before you start working with it.
</div>


### Description:
Nowadays, there are many healthcare provider companies that invest in technology based startups to assist the patients and doctors in diagnosis and curing process. You as a data scientist, are assigned to do a study on the data collected from 293 patients with heart disease, and extract some meaningfull information and report it to one of the healthcare provider companies. You can download the dataset (data.csv) from d2l and if you want to know more about it, you can use the below Kaggle link:<br>
https://www.kaggle.com/imnikhilanand/heart-attack-prediction



|Feature|Description|
|-------|----------------------------|
|age|age in years|
|gender|(1 = male; 0 = female)|
|cp|chest pain type|
|trestbps|resting blood pressure (in mm Hg on admission to the hospital)|
|chol|serum cholestoral in mg/dl|
|fbs|(fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)|
|restecg|resting electrocardiographic results|
|thalach|maximum heart rate achieved|
|exang|exercise induced angina (1 = yes; 0 = no)|
|oldpeak|ST depression induced by exercise relative to rest|
|slope|the slope of the peak exercise ST segment|
|ca|number of major vessels (0-3) colored by flourosopy|
|thal|3 = normal; 6 = fixed defect; 7 = reversable defect|
|num|diagnosis of heart disease (angiographic disease status)|

# Section A. Numpy (60pts)
### Part I. Table (40pts)
As a first step, to work properly with data, design a class that is responsible for general processes that you'll need.
* Keep header names in a python list, and store the the data rows in a numpy.ndarray as data attributes (why not class attribute ?)
* Implement a `readCSV` method which is responsible for reading header and data from a csv file.
* Implement a `printHead` method that print the header and first 10 rows of the table.
* Implement a `sort` method that sort the table based on the specified `column`. (`default='age'`)
* Implement a `deleteRow` method that gets a `row index` and deletes that row from the table. (`default=last row`) 
* Implement a `deleteCol` method that gets a `column index` and deletes that column from the table. (`default=last column`)
* Implement a `getColumn` method that gets a `column` name and after removing `nan` values from it, returns it as a numpy array. 
* Implement a `select` method that gets a `column` name and a `value`, and after removing the `nan` values, searchs for the records with `column=value` and returns that sub-table. 
* Implement a `rangeSelect` method that gets a `column` name and a `begin` and `end` (that define a range), and after removing the `nan` values and sorting the table based on `column`, searchs for the records with `begin<column<end` and returns that sub-table.
* Implement a `percentageSelect` method that gets a `column` name, a `Perc`(percentage) and `index`, and after removing the `nan` values and sorting the table based on `column` from that column: 
    * if index ==0 : returns the __first__ `Perc*column.size` sub-table.  
    * if index ==-1 : returns the __last__ `Perc*column.size` sub-table. 

<div class="alert alert-block alert-danger">
<b>Note:</b> <br>
Do not change the signature of methods and complete the same class structure as below<br>
`select`, `rangeSelect` and `percentageSelect` should return a new `Table` as their output without changing the current Table.<br>
To remove `nan` values and sort, you must reuse the `getColumn` and `sort` methods that you have implemented before.
</div>

In [3]:
import numpy as np
import csv

class Table:
    
    def __init__(self,header=[],data=np.array([])):
        self.header = header
        self.array = data
    
    def readCSV(self,filename:str)->None:
        self.array = np.genfromtxt(filename,dtype = float, delimiter = ',', names = True)
        with open(filename) as csvFile:
            reader = csv.reader(csvFile)
            self.header = next(reader)
        
    def printHead(self)->None:
        print(self.header)
        print(self.array[:10])
    
    def sort(self,column:str='age')->None:
        self.array.sort(order = column)

    def deleteRow(self,index:int=-1)->None:
        self.array = np.delete(self.array,index,axis = 0)

    def deleteCol(self,column:int=-1)->None:
        self.header = self.header[:column] + self.header[column+1:]
        self.array = self.array[self.header]

    def getColumn(self,column:str='age')->np.array:   
        newArray = self.array[column]
        cleanedNewArray = np.array([x for x in newArray if str(x) != 'nan'])
        return cleanedNewArray
        
    def select(self,column:str,value:float):
        filteredColumn = self.getColumn(column)
        filteredColumn = filteredColumn[filteredColumn == value]
        tempArray = np.in1d(self.array[column],filteredColumn)
        tempArray = self.array[tempArray]
        subTable = Table(self.header,tempArray)
        return subTable
    
    def rangeSelect(self,column:str,begin:int,end:int):
        self.sort(column)
        filteredColumn = self.getColumn(column)
        filteredColumn = filteredColumn[(filteredColumn < end) & (filteredColumn > begin)]
        tempArray = np.in1d(self.array[column],filteredColumn)
        tempArray = self.array[tempArray]
        subTable = Table(self.header,tempArray)
        return subTable

    def percentageSelect(self,column:str,Perc:float,index:int):
        self.sort(column)
        filteredColumn = self.getColumn(column)
        tempArray = np.in1d(self.array[column],filteredColumn)
        tempArray = self.array[tempArray]
        
        if index == -1:
            firstRow = tempArray.size - int(round(Perc*tempArray.size))
            tempArray = tempArray[firstRow:tempArray.size]
        elif index == 0:
            lastRow = int(round(Perc*tempArray.size))
            tempArray = tempArray[0:lastRow]
            
        subTable = Table(self.header,tempArray)
        return subTable 

In [6]:
# some test cases

"""  
make sure to add some more test cases to ensure the
correctness of all your methods before going to partII
"""

Synopsis=Table()
Synopsis.readCSV("data.csv")
Synopsis.printHead()
#Synopsis.sort(column = 'gender')
#Synopsis.getColumn('gender')
#Synopsis.printHead()
#Synopsis.select('age',58.0)
#Synopsis.rangeSelect('age',52,56)
# Synopsis.percentageSelect('thal',0.3,-1)
# Synopsis.printHead()
# Synopsis.sort(column='trestbps')
# Synopsis.printHead()
# Synopsis.deleteRow(0)
# Synopsis.printHead()
Synopsis.deleteCol(0)
Synopsis.printHead()

['age', 'gender', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num']
[(28., 1., 2., 130., 132., 0., 2., 185., 0., 0., nan, nan, nan, 0.)
 (29., 1., 2., 120., 243., 0., 0., 160., 0., 0., nan, nan, nan, 0.)
 (29., 1., 2., 140.,  nan, 0., 0., 170., 0., 0., nan, nan, nan, 0.)
 (30., 0., 1., 170., 237., 0., 1., 170., 0., 0., nan, nan,  6., 0.)
 (31., 0., 2., 100., 219., 0., 1., 150., 0., 0., nan, nan, nan, 0.)
 (32., 0., 2., 105., 198., 0., 0., 165., 0., 0., nan, nan, nan, 0.)
 (32., 1., 2., 110., 225., 0., 0., 184., 0., 0., nan, nan, nan, 0.)
 (32., 1., 2., 125., 254., 0., 0., 155., 0., 0., nan, nan, nan, 0.)
 (33., 1., 3., 120., 298., 0., 0., 185., 0., 0., nan, nan, nan, 0.)
 (34., 0., 2., 130., 161., 0., 0., 190., 0., 0., nan, nan, nan, 0.)]
['gender', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num']
[(1., 2., 130., 132., 0., 2., 185., 0., 0., nan, nan, nan, 0.)
 (1., 2., 120., 2

In [130]:
# Your Code goes here - complete the code bellow

report=Table()
report.readCSV("data.csv")

# report 1
ageList = report.getColumn('age')
total = 0
for x in ageList:
    total += x
avgAge = total/ageList.size
print("Report1\n \tthe average age of patients are: ", avgAge)

# report2
print("\nReport2")
for i in range(20,60,10):
    subTable = report.rangeSelect('age',i,i+10.0)
    subColumn = subTable.getColumn('chol')
    total = 0
    for x in subColumn:
        total += x
    avgChol = total/subColumn.size
    print("\tfor people with age in range of", i,i+10,"average CHOL is: ",avgChol)

# report3
print("\nReport3")
subTableLow = report.percentageSelect('chol',0.3,0)
subTableHigh = report.percentageSelect('chol',0.3,-1)

subColumnLow = subTableLow.getColumn('trestbps')
lowTotal = 0
for x in subColumnLow:
    lowTotal += x
lowAvg = lowTotal/subColumnLow.size

subColumnHigh = subTableHigh.getColumn('trestbps')
highTotal = 0
for x in subColumnHigh:
    highTotal += x
highAvg = highTotal/subColumnHigh.size
print("\tfor patients with lowest 30% of chol, average trestbps is: ",lowAvg)
print("\tfor patients with highest 30% of chol, average trestbps is: ",highAvg) 

# report4
print("\nReport4")
subTablePositive = report.select('num',1)
subColumnPositive = subTablePositive.getColumn('gender')
menTotal = 0
womenTotal = 0
for x in subColumnPositive:
    if x == 1:
        menTotal += 1
    elif x == 0:
        womenTotal += 1
menPercent = menTotal/(menTotal + womenTotal)*100
womenPercent = womenTotal/(menTotal + womenTotal)*100

print("\t{0:.2f}% of patients diagnosed with heart decease are men and ".format(menPercent),"{0:.2f}% of them are women".format(womenPercent))


Report1
 	the average age of patients are:  47.767918088737204

Report2
	for people with age in range of 20 30 average CHOL is:  187.5
	for people with age in range of 30 40 average CHOL is:  239.6888888888889
	for people with age in range of 40 50 average CHOL is:  245.6451612903226
	for people with age in range of 50 60 average CHOL is:  258.6666666666667

Report3
	for patients with lowest 30% of chol, average trestbps is:  132.97530864197532
	for patients with highest 30% of chol, average trestbps is:  135.6375

Report4
	88.57% of patients diagnosed with heart decease are men and  11.43% of them are women


## Section B. Pandas (40pts)

Now generate the same reports(sectionA.partII) again, but this time use Pandas dataframe instead of `Table` class. 

In [8]:
# Your Code goes here - complete the code bellow

import pandas as pd

df = pd.read_csv('data.csv', index_col=False, header=0,na_values="?")
print(df.head())

# report 1
avgAge = df['age'].mean()
print("Report1\n \tthe average age of patients are: ",avgAge)

# report2
print("\nReport2")
for i in range(20,60,10):
    avgChol = df[(df['age']>i)&(df['age']<i+10)]['chol'].mean(axis = 0)
    print("\tfor people with age in range of", i,i+10,"average CHOL is: ",avgChol)

# report3
print("\nReport3")
sorted_df = df.sort_values('chol')
numRows = int(round(0.3*len(sorted_df)))

lowAvg = sorted_df['trestbps'].head(numRows).mean()
highAvg = sorted_df['trestbps'].tail(numRows).mean()

print("\tfor patients with lowest 30% of chol, average trestbps is: ",lowAvg)
print("\tfor patients with highest 30% of chol, average trestbps is: ",highAvg) 

# report4
print("\nReport4")
menTotal = len(df[(df['num']==1)&(df['gender']==1)])
womenTotal = len(df[(df['num']==1)&(df['gender']==0)])
menPercent = menTotal/(menTotal+womenTotal)*100
womenPercent = womenTotal/(menTotal+womenTotal)*100
print("\t{0:.2f}% of patients diagnosed with heart decease are men and ".format(menPercent),"{0:.2f}% of them are women".format(womenPercent))


   age  gender  cp  trestbps   chol  fbs  restecg  thalach  exang  oldpeak  \
0   28       1   2     130.0  132.0  0.0      2.0    185.0    0.0      0.0   
1   29       1   2     120.0  243.0  0.0      0.0    160.0    0.0      0.0   
2   29       1   2     140.0    NaN  0.0      0.0    170.0    0.0      0.0   
3   30       0   1     170.0  237.0  0.0      1.0    170.0    0.0      0.0   
4   31       0   2     100.0  219.0  0.0      1.0    150.0    0.0      0.0   

   slope  ca  thal  num  
0    NaN NaN   NaN    0  
1    NaN NaN   NaN    0  
2    NaN NaN   NaN    0  
3    NaN NaN   6.0    0  
4    NaN NaN   NaN    0  
Report1
 	the average age of patients are:  47.767918088737204

Report2
	for people with age in range of 20 30 average CHOL is:  187.5
	for people with age in range of 30 40 average CHOL is:  239.6888888888889
	for people with age in range of 40 50 average CHOL is:  245.6451612903226
	for people with age in range of 50 60 average CHOL is:  258.6666666666667

Report3
	for p